In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.utils import get_device

import matplotlib.pyplot as plt
import numpy as np

/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_convnext.py')

root='/data/ephemeral/home/dataset/'

cfg.keys()

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'albu_train_transforms', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'opencv_num_threads', 'mp_start_method', 'auto_scale_lr', 'custom_imports', 'checkpoint_file'])

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train_split.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (1024, 1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'val_split.json' # train json 정보'
cfg.data.val.pipeline[1]['img_scale'] = (1024, 1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (1024, 1024) # Resize

cfg.data.samples_per_gpu = 2 # 한 GPU가 학습 시 한 번에 처리할 이미지(샘플)의 개수(배치 크기 조절)
cfg.data.workers_per_gpu = 4 # 한 GPU가 데이터를 불러올 때 병렬로 작업할 워커(worker)의 수

cfg.seed = 2024
cfg.gpu_ids = [0]
# cfg.work_dir = './work_dirs/custom_cascade(Shear,Rotate,Translate,RandomShift,MinIoURandomCrop)_log'
cfg.work_dir = './work_dirs/cascade_rcnn_convnext_(cutout_albu)_log'

# cfg.model.roi_head.bbox_head.num_classes = 10
for i in range(3):
    cfg.model.roi_head.bbox_head[i].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2) # 학습 중 기울기 폭발을 방지하기 위해 사용
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1) # 최대 3개의 체크포인트 파일을 유지

cfg.log_config = dict(
    interval=50,  # 로그를 50 iteration마다 출력
    hooks=[
        dict(type='TextLoggerHook'),  # 기본 Text Logger
        dict(type='TensorboardLoggerHook'),  # Tensorboard Logger 추가
    ])

cfg.evaluation = dict(
    interval=1,
    metric='bbox',
    save_best='auto',
    by_epoch=True
)

cfg.device = get_device()

In [4]:
# build_dataset

datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [5]:
datasets[0]


CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
datasets[0][0]

{'img_metas': DataContainer({'filename': '/data/ephemeral/home/dataset/train/0000.jpg', 'ori_filename': 'train/0000.jpg', 'ori_shape': (1024, 1024, 3), 'img_shape': (1024, 1024, 3), 'pad_shape': (1024, 1024, 3), 'scale_factor': array([1., 1., 1., 1.], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}),
 'img': DataContainer(tensor([[[-0.7650, -0.4911, -0.3883,  ..., -0.9363, -0.9363, -0.7650],
          [ 0.8447,  0.1426, -0.6623,  ..., -0.7993, -0.8335, -0.7137],
          [ 0.6734,  0.5878, -0.2513,  ..., -0.6794, -0.7479, -0.6965],
          ...,
          [ 0.0056, -0.6794, -0.7137,  ..., -0.8678, -0.8678, -0.8678],
          [-0.2342, -0.7479, -0.6281,  ..., -0.7650, -0.8335, -0.9192],
          [-0.5082, -1.0048, -0.7479,  ..., -0.6794, -0.8335, -0.9705]],
 
         [[-0.7227, -0.4601, -0.3550,  ..., -0.7577, -0.7577, -0.5826],


시각화

In [ ]:
def visualize_selected_images(dataset, indices, cols=5):
    """선택된 인덱스에 해당하는 이미지와 바운딩 박스를 시각화."""
    num_images = len(indices)
    rows = num_images // cols + (num_images % cols > 0)

    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))  # 5x4 형태로 시각화
    axes = axes.flatten()

    for i, idx in enumerate(indices):
        # 해당 인덱스 데이터 선택
        data = dataset[idx]

        img = data['img'].data.permute(1, 2, 0).numpy()  # (C, H, W) -> (H, W, C)
        img = img[..., ::-1]  # BGR -> RGB 변환
        img = img.astype(np.uint8)

        gt_bboxes = data['gt_bboxes'].data.numpy()
        gt_labels = data['gt_labels'].data.numpy()

        # 이미지 경로에서 파일명 추출
        img_path = data['img_metas'].data['filename']
        img_name = img_path.split('/')[-1]  # '/'로 split해서 마지막 부분 추출

        # 현재 subplot에 이미지 표시
        ax = axes[i]
        ax.imshow(img)
        ax.axis('off')

        # 바운딩 박스 및 라벨 표시
        for bbox, label in zip(gt_bboxes, gt_labels):
            x1, y1, x2, y2 = bbox
            ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                       edgecolor='red', linewidth=2, fill=False))
            # 바운딩 박스 중앙에 라벨 표시
            ax.text(x1, y1 - 5, f'Label: {label}', color='yellow', fontsize=8, va='bottom')

        # 제목으로 이미지 파일명 표시
        ax.set_title(f"{img_name}")

    # 남은 빈 subplot 숨기기
    for j in range(num_images, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# 원하는 인덱스 20개 지정
indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

# 선택된 인덱스의 이미지 시각화
visualize_selected_images(datasets[0], indices=indices)


In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-18 14:24:54,943 - mmcv - INFO - initialize ConvNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/convnext/convnext-large_in21k-pre-3rdparty_64xb64_in1k_20220124-2412403d.pth', 'prefix': 'backbone.'}
2024-10-18 14:24:54,944 - mmcv - INFO - load backbone. in model from: https://download.openmmlab.com/mmclassification/v0/convnext/convnext-large_in21k-pre-3rdparty_64xb64_in1k_20220124-2412403d.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/convnext/convnext-large_in21k-pre-3rdparty_64xb64_in1k_20220124-2412403d.pth


2024-10-18 14:24:55,666 - mmcv - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias

2024-10-18 14:24:55,797 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-18 14:24:55,821 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-18 14:24:55,832 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2024-10-18 14:24:55,953 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'overrid

In [8]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-10-18 14:24:58,227 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-18 14:24:58,290 - mmdet - INFO - Start running, host: root@instance-12326, work_dir: /data/ephemeral/home/git/level2-objectdetection-cv-03/mmdetection_base/work_dirs/cascade_rcnn_convnext_(cutout_albu)_log
2024-10-18 14:24:58,291 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerH

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2024-10-18 14:25:58,793 - mmdet - INFO - Epoch [1][50/1957]	lr: 1.978e-05, eta: 13:07:14, time: 1.208, data_time: 0.056, memory: 19245, loss_rpn_cls: 0.5907, loss_rpn_bbox: 0.0426, s0.loss_cls: 0.8244, s0.acc: 80.8828, s0.loss_bbox: 0.0575, s1.loss_cls: 0.4294, s1.acc: 80.0254, s1.loss_bbox: 0.0159, s2.loss_cls: 0.1802, s2.acc: 85.7773, s2.loss_bbox: 0.0022, loss: 2.1430, grad_norm: 20.4017
2024-10-18 14:26:55,481 - mmdet - INFO - Epoch [1][100/1957]	lr: 3.976e-05, eta: 12:41:56, time: 1.134, data_time: 0.007, memory: 19245, loss_rpn_cls: 0.2016, loss_rpn_bbox: 0.0474, s0.loss_cls: 0.3741, s0.acc: 93.7168, s0.loss_bbox: 0.1469, s1.loss_cls: 0.1116, s1.acc: 97.1875, s1.loss_bbox: 0.0421, s2.loss_cls: 0.0384, s2.acc: 98.4961, s2.loss_bbox: 0.0057, loss: 0.9679, grad_norm: 4.6203
2024-10-18 14:27:52,054 - mmdet - INFO - Epoch [1][150/1957]	lr: 5.974e-05, eta: 12:32:23, time: 1.131, data_time: 0.007, memory: 19245, loss_rpn_cls: 0.1458, loss_rpn_bbox: 0.0497, s0.loss_cls: 0.3183, s0.acc: 9

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 969/969, 5.1 task/s, elapsed: 192s, ETA:     0s

2024-10-18 15:05:20,826 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 969/969, 5.1 task/s, elapsed: 192s, ETA:     0s

2024-10-19 03:55:12,555 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.93s).
Accumulating evaluation results...


2024-10-19 03:55:15,117 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.461
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.576
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.492
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.536
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.553
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.553
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.553
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.033
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.165
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.41s).
